In [1]:
# Importing necessary libraries

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

In [2]:
# Importing labelled dataset from the detection module

df = pd.read_csv('D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/labelledData.csv')

In [3]:
# Correcting data-type of categorical features and dropping few irrelevant like: 'Anomaly_Score','Unnamed: 0','timestamps'

df['BMS_max_cell_temp_id'] = df['BMS_max_cell_temp_id'].astype('object')
df['BMS_state'] = df['BMS_state'].astype('object')
df['BMS_min_cell_temp_id'] = df['BMS_min_cell_temp_id'].astype('object')
df['BMS_max_cell_voltage_id'] = df['BMS_max_cell_voltage_id'].astype('object')
df['BMS_min_cell_voltage_id'] = df['BMS_min_cell_voltage_id'].astype('object')
df['OBC_mux'] = df['OBC_mux'].astype('object')
df['OBC_port_status'] = df['OBC_port_status'].astype('object')
df['OBC_overvoltage_fault'] = df['OBC_overvoltage_fault'].astype('object')
df['OBC_overcurrent_fault'] = df['OBC_overcurrent_fault'].astype('object')
df['OBC_port_weld_fault'] = df['OBC_port_weld_fault'].astype('object')
df['Anomaly'] = df['Anomaly'].astype('object')

df = df.drop(['Anomaly_Score','Unnamed: 0','timestamps'], axis = 1)

In [4]:
# Isolating predictor features from target feature

x = df.drop('Anomaly', axis =1)
y = df['Anomaly'].astype('int')

In [5]:
# Isolating Numerical & Categorical variables

char = []
num = []

for i in x.columns:
    if x[i].dtype == 'object':
        char.append(i)
    else:
        num.append(i)

In [6]:
# Numeric transformations: Impute missing values and scaling

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical transformations: Impute missing and OneHotEncode

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

In [7]:
# Use ColumnTransformer to apply transformations

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num),
        ('cat', categorical_transformer, char)
    ])

In [8]:
# Create the final pipeline

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [9]:
# Performing RandomOverSampling to address Imbalance data

sampler = RandomOverSampler(random_state=0,sampling_strategy='auto')
x_resampled, y_resampled = sampler.fit_resample(x,y)

In [10]:
# Train test split

x_train, x_test,y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.3, random_state=101)

In [11]:
# Fit the pipeline on the training data
pipeline.fit(x_train, (y_train))

# Make predictions on the test data
y_pred = pipeline.predict(x_test)

In [12]:
# Checking accuracy on test data

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.9985786802030456


In [13]:
# Save the preprocessor
joblib.dump(preprocessor,'D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/artifacts/Preprocessor.pkl')

# Save the model
joblib.dump(pipeline, 'D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/artifacts/Model.pkl')

['D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/artifacts/Model.pkl']